In [1]:
import pandas as pd
import json

# Leer el JSON
with open("full-data_zonaprop.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convertir a DataFrame
df = pd.DataFrame(data)



In [2]:
#df = df.drop(columns=["error", "raw"], errors="ignore")

In [3]:
#df = df.loc[:, df.notna().sum() > 10]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Titulo       1884 non-null   object
 1   Operacion    1884 non-null   object
 2   Direccion    1884 non-null   object
 3   Ubicacion    1884 non-null   object
 4   Precio       1884 non-null   object
 5   Total        1864 non-null   object
 6   Cubierta     1862 non-null   object
 7   Ambientes    1873 non-null   object
 8   Baños        1859 non-null   object
 9   Dormitorios  1510 non-null   object
 10  Antigüedad   1847 non-null   object
 11  Disposición  1530 non-null   object
 12  Orientación  1052 non-null   object
 13  Luminosidad  897 non-null    object
 14  Cocheras     581 non-null    object
 15  Toilettes    711 non-null    object
dtypes: object(16)
memory usage: 278.4+ KB


In [5]:
df["Direccion"] = df["Direccion"].str.split(",").str[0].str.strip()

In [6]:
#df.rename(columns={
#    "titulo": "Titulo",
#    "operacion": "Operacion",
#    "direccion": "Direccion",
#    "https": "Inmobiliaria"
#}, inplace=True)

In [7]:
#df.drop(columns=["http"], inplace=True)

In [8]:
df['Barrio'] = "Palermo"

In [9]:
"""
# Separar ubicacion por coma
df[['barrio', 'ciudad']] = df['ubicacion'].str.split(',', expand=True)[[1, 2]]

# Limpiar espacios
df['barrio'] = df['barrio'].str.strip()
df['ciudad'] = df['ciudad'].str.strip()
df = df.drop(columns=["ubicacion"])
"""

'\n# Separar ubicacion por coma\ndf[[\'barrio\', \'ciudad\']] = df[\'ubicacion\'].str.split(\',\', expand=True)[[1, 2]]\n\n# Limpiar espacios\ndf[\'barrio\'] = df[\'barrio\'].str.strip()\ndf[\'ciudad\'] = df[\'ciudad\'].str.strip()\ndf = df.drop(columns=["ubicacion"])\n'

In [10]:
"""
df['fecha_publicacion'] = df['fecha_publicacion'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], dayfirst=True)
"""

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cacic\AppData\Local\Temp\ipykernel_1896\1555651723.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


"\ndf['fecha_publicacion'] = df['fecha_publicacion'].str.extract(r'(\\d{1,2}/\\d{1,2}/\\d{4})')\ndf['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], dayfirst=True)\n"

In [11]:
import re

def limpiar_direccion(direccion):
    if pd.isna(direccion):
        return direccion
    
    # Convertir a string por si acaso
    direccion = str(direccion)
    
    # 1. Eliminar todo lo que está entre paréntesis (incluyendo los paréntesis)
    direccion = re.sub(r'\([^)]*\)', '', direccion)
    
    # 2. Eliminar lo que está después de ciertos separadores
    direccion = direccion.split(' -')[0]  # después de guión
    direccion = direccion.split(' y')[0]  # después de 'y'
    direccion = direccion.split(',')[0]   # después de coma
    
    # 3. Correcciones específicas conocidas
    calle_reemplazos = {
        r'\bOro\b': 'Fray Justo Santa María de Oro',
        r'\bSta\.?\b': 'Santa',
        r'\bSta María\b': 'Santa María',
        r"\b(Al\s+){2,}": " ",
        r"\b(al\s+){2,}": " ",
    }
    
    for patron, reemplazo in calle_reemplazos.items():
        direccion = re.sub(patron, reemplazo, direccion, flags=re.IGNORECASE)
    
    # 4. Limpieza final - eliminar espacios múltiples y trim
    direccion = re.sub(r'\s+', ' ', direccion).strip()
    
    return direccion

In [12]:
# Aplicar la función a tu DataFrame
df['Direccion'] = df['Direccion'].apply(limpiar_direccion)

df['Direccion'] = df['Direccion'].str.upper()

# Mostrar algunos ejemplos
print(df[['Direccion', 'Direccion']].head())

                                Direccion  \
0                       GUATEMALA AL 4500   
1                        MANSILLA AL 4000   
2  VENTA DEPARTAMENTO PALERMO LAS CAÑITAS   
3                                     NaN   
4        DEPARTAMENTO EN VENTA EN PALERMO   

                                Direccion  
0                       GUATEMALA AL 4500  
1                        MANSILLA AL 4000  
2  VENTA DEPARTAMENTO PALERMO LAS CAÑITAS  
3                                     NaN  
4        DEPARTAMENTO EN VENTA EN PALERMO  


In [13]:
#1
df["Direccion"] = df["Direccion"].str.replace(r"\s*al\s+", " ", regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'Entre\s+[^0-9]*\s+[Yy]\s+[^0-9]*', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)\b(av\.?|avenida)\b'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace('.', '')
df['Direccion'] = df['Direccion'].str.replace(',', '')
df['Direccion'] = df['Direccion'].str.replace(r'(?i)\s+y\s+[^-\d]+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*piso\s*\d+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*unidad\s*\w+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*cfrt'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*uf\s*\d+', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+\s*amb', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+°?º?p', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*frt', " ",regex=True) 
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+°?\s*piso', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r',\s*C?\d{4}.*', '', regex=True)


In [14]:
df

,Titulo,Operacion,Direccion,Ubicacion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,"-34.587780600000002, -58.422817100000003",USD 268.250,68 m²,63 m²,3,2,2,0 años,NaN,NaN,NaN,NaN,NaN,Palermo
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,"-34.587902100000000, -58.418962700000001",USD 89.000,31 m²,31 m²,2,1,1,0 años,NaN,NaN,NaN,NaN,NaN,Palermo
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,"-34.568259595900002, -58.444749263600002",USD 148.000,38 m²,38 m²,1,1,NaN,0 años,Frente,N,Muy luminoso,NaN,NaN,Palermo
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palermo
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,"-34.578065000000002, -58.426530999999997",USD 190.000,63 m²,58 m²,3,1,2,0 años,Contrafrente,N,NaN,NaN,NaN,Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,"-34.582848300000001, -58.433264000000001",USD 470.000,108 m²,96 m²,3,2,2,2 años,Contrafrente,NO,Muy luminoso,1,NaN,Palermo
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,"-34.579716699999998, -58.413954300000000",USD 753.600,142 m²,120 m²,5,4,3,0 años,Frente,NaN,Muy luminoso,2,NaN,Palermo
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,"-34.581283499999997, -58.442193500000001",USD 179.900,55 m²,55 m²,2,NaN,1,0 años,Frente,NaN,NaN,1,1,Palermo
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,"-34.590427599999998, -58.424701300000002",USD 236.500,105 m²,45 m²,2,1,1,0 años,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo


In [15]:
def limpiar_columnas_numericas(df, columnas):
    for columna in columnas:
        if columna in df.columns:
            # Extrae solo los números, incluyendo puntos
            df[columna] = df[columna].astype(str).str.extract(r'(\d[\d\.]*)')
            # Convierte a tipo numérico
            df[columna] = pd.to_numeric(df[columna], errors='coerce')
    return df

In [16]:
df = limpiar_columnas_numericas(df, ["Precio", "Total", "Cubierta","Ambientes", "Baños", "Antigüedad", "Dormitorios"])
df

,Titulo,Operacion,Direccion,Ubicacion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,"-34.587780600000002, -58.422817100000003",268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,"-34.587902100000000, -58.418962700000001",89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,"-34.568259595900002, -58.444749263600002",148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palermo
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,"-34.578065000000002, -58.426530999999997",190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,"-34.582848300000001, -58.433264000000001",470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,"-34.579716699999998, -58.413954300000000",753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,"-34.581283499999997, -58.442193500000001",179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,"-34.590427599999998, -58.424701300000002",236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo


In [17]:
def categorizar_antiguedad(antiguedad_str):
    try:
        # Extraer solo los números (incluyendo decimales)
        antiguedad_num = float(re.sub(r'[^\d.]', '', str(antiguedad_str)))
    except:
        return None
    
    # Redondear al entero más cercano
    antiguedad = int(round(antiguedad_num))
    
    if antiguedad == 0:
        return 0  # A estrenar
    elif antiguedad <= 5:
        return 1  # Muy nueva
    elif antiguedad <= 10:
        return 2  # Nueva
    elif antiguedad <= 20:
        return 3  # Relativamente nueva
    elif antiguedad <= 40:
        return 4  # Media
    else:
        return 5  # Antigua

# Aplicar la función
df["Categoria_antigüedad"] = df["Antigüedad"].apply(categorizar_antiguedad)

df

,Titulo,Operacion,Direccion,Ubicacion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,"-34.587780600000002, -58.422817100000003",268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,"-34.587902100000000, -58.418962700000001",89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,"-34.568259595900002, -58.444749263600002",148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palermo,NaN
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,"-34.578065000000002, -58.426530999999997",190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,"-34.582848300000001, -58.433264000000001",470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,"-34.579716699999998, -58.413954300000000",753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,"-34.581283499999997, -58.442193500000001",179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,"-34.590427599999998, -58.424701300000002",236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0


In [18]:
df['Publicada_en']="ZonaProp"

df

,Titulo,Operacion,Direccion,Ubicacion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,"-34.587780600000002, -58.422817100000003",268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,"-34.587902100000000, -58.418962700000001",89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,"-34.568259595900002, -58.444749263600002",148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palermo,NaN,ZonaProp
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,"-34.578065000000002, -58.426530999999997",190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,"-34.582848300000001, -58.433264000000001",470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,"-34.579716699999998, -58.413954300000000",753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,"-34.581283499999997, -58.442193500000001",179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,"-34.590427599999998, -58.424701300000002",236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp


In [19]:
def explotar_ubicacion(df):
    # Verifica que la columna exista
    if "Ubicacion" not in df.columns:
        raise ValueError("La columna 'Ubicacion' no existe en el DataFrame.")
    
    # Divide la columna en dos listas
    latitudes = df["Ubicacion"].str.split(",", n=1).str[0].str.strip().tolist()
    longitudes = df["Ubicacion"].str.split(",", n=1).str[1].str.strip().tolist()
    
    # Agrega las columnas al DataFrame
    df["lat"] = latitudes
    df["lon"] = longitudes

    # Elimina la columna original
    df = df.drop(columns=["Ubicacion"])
    
    return df


In [20]:
df = explotar_ubicacion(df)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Titulo                1884 non-null   object 
 1   Operacion             1884 non-null   object 
 2   Direccion             1884 non-null   object 
 3   Precio                1761 non-null   float64
 4   Total                 1864 non-null   float64
 5   Cubierta              1862 non-null   float64
 6   Ambientes             1873 non-null   float64
 7   Baños                 1859 non-null   float64
 8   Dormitorios           1510 non-null   float64
 9   Antigüedad            1838 non-null   float64
 10  Disposición           1530 non-null   object 
 11  Orientación           1052 non-null   object 
 12  Luminosidad           897 non-null    object 
 13  Cocheras              581 non-null    object 
 14  Toilettes             711 non-null    object 
 15  Barrio               

In [23]:
print("\nDespués de la limpieza:")
print("Valores únicos en 'Total':", df['Total'].unique())
print("Cantidad de NaN:", df['Total'].isna().sum())
print("\nPrimeras filas:")
print(df[['Titulo', 'Total']].head())


Después de la limpieza:
Valores únicos en 'Total': [ 68.  31.  38.  nan  63. 162.  60. 105.  65.  37.  88.  56. 154.  73.
  40.  32. 181.  48.  22.  44.  28.  50.  46.  42.  33.  72.  39. 342.
  30.  43. 195. 125.  45.  29.  54. 101.  49.  41. 396.  55.  34.  47.
 142. 227.  78.  53.  61. 114.  70.  52.  69. 133.  19.  26.  36.  64.
 121.  95.  51.  57.  75.  80.  92. 455.  81. 103.  66. 351. 490. 177.
  67. 107. 110. 120. 252. 148.  94. 240. 200. 119.  74.  82. 242.  35.
 243. 205. 118. 207.  83. 239. 282. 150.  99.  79. 283.  76. 226. 108.
 245.  85. 111.  58.  84. 145. 182. 213.  90. 178. 152. 183. 180. 122.
  93. 130. 217. 175. 450.  98.  59.  18. 166. 160. 247. 116. 506. 487.
  89. 156. 186. 144. 190. 131. 141. 197. 135.  77. 140. 102. 253. 193.
 251.  87. 222. 165.  91. 100. 232. 255. 134. 250.  62. 198. 215.  71.
 307. 132.  25. 258. 330. 287. 155.  21. 392. 362. 265. 290. 357. 188.
 187. 202. 400. 138. 466.  96.  86. 210. 173. 157. 556. 113. 327. 858.
 502. 164. 112. 115. 206.

In [24]:
# Eliminar filas donde Precio, lon o lat sean NaN
df_clean = df.dropna(subset=['Precio', 'Total', 'lon', 'lat'], how='any')

In [25]:
print("Valores únicos en 'Total' después de la limpieza:")
print(df['Total'].value_counts(dropna=False))

print("\nCantidad de ceros restantes:", (df['Total'] == 0).sum())

Valores únicos en 'Total' después de la limpieza:
Total
NaN      362
40.0      48
34.0      40
47.0      34
52.0      31
        ... 
309.0      1
168.0      1
600.0      1
268.0      1
452.0      1
Name: count, Length: 263, dtype: int64

Cantidad de ceros restantes: 0


In [26]:
df

,Titulo,Operacion,Direccion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587780600000002,-58.422817100000003
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587902100000000,-58.418962700000001
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.568259595900002,-58.444749263600002
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Palermo,NaN,ZonaProp,NaN,NaN
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.578065000000002,-58.426530999999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp,-34.582848300000001,-58.433264000000001
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp,-34.579716699999998,-58.413954300000000
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp,-34.581283499999997,-58.442193500000001
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.590427599999998,-58.424701300000002


In [27]:
df_clean

,Titulo,Operacion,Direccion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587780600000002,-58.422817100000003
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587902100000000,-58.418962700000001
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.568259595900002,-58.444749263600002
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.578065000000002,-58.426530999999997
5,Departamento · 146m² · 4 ambientes · 1 cochera,venta,DEPTO EN VENTA,500.00,162.0,146.0,4.0,2.0,2.0,19.0,Contrafrente,NE,NaN,1,1,Palermo,3.0,ZonaProp,-34.581402500000002,-58.441511499999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp,-34.582848300000001,-58.433264000000001
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp,-34.579716699999998,-58.413954300000000
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp,-34.581283499999997,-58.442193500000001
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.590427599999998,-58.424701300000002


In [28]:
conteo_ceros = df_clean['Total'].value_counts().get(0, 0)
print(f"Cantidad de registros con total = 0: {conteo_ceros}")

Cantidad de registros con total = 0: 0


In [29]:
nan_precio = df_clean["Precio"].isna().sum()
print(f"Número de propiedades sin precio: {nan_precio}")

nan_Total = df_clean["Total"].isna().sum()
print(f"Número de propiedades sin Total: {nan_Total}")

Número de propiedades sin precio: 0
Número de propiedades sin Total: 0


In [30]:
def limpiar_columnas_numericas(df, columnas):
    for columna in columnas:
        if columna in df.columns:
            df[columna] = (
                df[columna]
                .astype(str)
                .str.replace(r'[^\d.,]', '', regex=True)  # dejar números, comas, puntos
                .str.replace(',', '.', regex=False)       # estandarizar decimales
            )
            # Quitar punto si actúa como separador de miles (solo si hay más de un punto)
            df[columna] = df[columna].str.replace(r'(?<=\d)\.(?=\d{3}(?:\.|$))', '', regex=True)
            df[columna] = pd.to_numeric(df[columna], errors='coerce')
    return df

In [31]:
copia = df_clean.copy()

In [32]:
copia

,Titulo,Operacion,Direccion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587780600000002,-58.422817100000003
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587902100000000,-58.418962700000001
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.568259595900002,-58.444749263600002
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.578065000000002,-58.426530999999997
5,Departamento · 146m² · 4 ambientes · 1 cochera,venta,DEPTO EN VENTA,500.00,162.0,146.0,4.0,2.0,2.0,19.0,Contrafrente,NE,NaN,1,1,Palermo,3.0,ZonaProp,-34.581402500000002,-58.441511499999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp,-34.582848300000001,-58.433264000000001
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp,-34.579716699999998,-58.413954300000000
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp,-34.581283499999997,-58.442193500000001
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.590427599999998,-58.424701300000002


In [33]:
copia = limpiar_columnas_numericas(copia, ["Total", "Cubierta", "Ambientes", "Baños", "Antigüedad", "Dormitorios"])

copia

,Titulo,Operacion,Direccion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,268.25,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587780600000002,-58.422817100000003
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,89.00,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587902100000000,-58.418962700000001
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,148.00,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.568259595900002,-58.444749263600002
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,190.00,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.578065000000002,-58.426530999999997
5,Departamento · 146m² · 4 ambientes · 1 cochera,venta,DEPTO EN VENTA,500.00,162.0,146.0,4.0,2.0,2.0,19.0,Contrafrente,NE,NaN,1,1,Palermo,3.0,ZonaProp,-34.581402500000002,-58.441511499999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,470.00,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp,-34.582848300000001,-58.433264000000001
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,753.60,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp,-34.579716699999998,-58.413954300000000
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,179.90,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp,-34.581283499999997,-58.442193500000001
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,236.50,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.590427599999998,-58.424701300000002


In [34]:
copia["Precio"]

0       268.25
1        89.00
2       148.00
4       190.00
5       500.00
         ...  
2221    470.00
2222    753.60
2223    179.90
2224    236.50
2225    170.00
Name: Precio, Length: 1741, dtype: float64

In [35]:
# Solución correcta para tu caso
copia["Precio"] = (copia["Precio"] * 1000).astype(int)

copia["Precio"]

0       268250
1        89000
2       148000
4       190000
5       500000
         ...  
2221    470000
2222    753600
2223    179900
2224    236500
2225    170000
Name: Precio, Length: 1741, dtype: int64

In [36]:

copia


,Titulo,Operacion,Direccion,Precio,Total,Cubierta,Ambientes,Baños,Dormitorios,Antigüedad,Disposición,Orientación,Luminosidad,Cocheras,Toilettes,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Departamento · 63m² · 3 ambientes,venta,GUATEMALA AL 4500,268250,68.0,63.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587780600000002,-58.422817100000003
1,Departamento · 31m² · 2 ambientes,venta,MANSILLA AL 4000,89000,31.0,31.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.587902100000000,-58.418962700000001
2,Departamento · 38m² · 1 ambiente,venta,VENTA DEPARTAMENTO PALERMO LAS CAÑITAS,148000,38.0,38.0,1.0,1.0,NaN,0.0,Frente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.568259595900002,-58.444749263600002
4,Departamento · 58m² · 3 ambientes,venta,DEPARTAMENTO EN VENTA EN PALERMO,190000,63.0,58.0,3.0,1.0,2.0,0.0,Contrafrente,N,NaN,NaN,NaN,Palermo,0.0,ZonaProp,-34.578065000000002,-58.426530999999997
5,Departamento · 146m² · 4 ambientes · 1 cochera,venta,DEPTO EN VENTA,500000,162.0,146.0,4.0,2.0,2.0,19.0,Contrafrente,NE,NaN,1,1,Palermo,3.0,ZonaProp,-34.581402500000002,-58.441511499999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,Departamento · 96m² · 3 ambientes · 1 cochera,venta,HUMBOLDT AL 1900,470000,108.0,96.0,3.0,2.0,2.0,2.0,Contrafrente,NO,Muy luminoso,1,NaN,Palermo,1.0,ZonaProp,-34.582848300000001,-58.433264000000001
2222,Departamento · 120m² · 5 ambientes · 2 cocheras,venta,CABELLO AL 3800,753600,142.0,120.0,5.0,4.0,3.0,0.0,Frente,NaN,Muy luminoso,2,NaN,Palermo,0.0,ZonaProp,-34.579716699999998,-58.413954300000000
2223,Departamento · 55m² · 2 ambientes · 1 cochera,venta,GORRITI AL 6000,179900,55.0,55.0,2.0,NaN,1.0,0.0,Frente,NaN,NaN,1,1,Palermo,0.0,ZonaProp,-34.581283499999997,-58.442193500000001
2224,Departamento · 45m² · 2 ambientes,venta,SCALABRINI ORTIZ AL 1600,236500,105.0,45.0,2.0,1.0,1.0,0.0,Contrafrente,N,Muy luminoso,NaN,NaN,Palermo,0.0,ZonaProp,-34.590427599999998,-58.424701300000002


In [37]:

sorted(copia["Precio"].unique())


[np.int64(48000),
 np.int64(57000),
 np.int64(59000),
 np.int64(59900),
 np.int64(64000),
 np.int64(65000),
 np.int64(67000),
 np.int64(69000),
 np.int64(69900),
 np.int64(70000),
 np.int64(71000),
 np.int64(72000),
 np.int64(73000),
 np.int64(73200),
 np.int64(74000),
 np.int64(74600),
 np.int64(75000),
 np.int64(76000),
 np.int64(76900),
 np.int64(77000),
 np.int64(77900),
 np.int64(78000),
 np.int64(79000),
 np.int64(79800),
 np.int64(79900),
 np.int64(80000),
 np.int64(80500),
 np.int64(81000),
 np.int64(82000),
 np.int64(83000),
 np.int64(83500),
 np.int64(83720),
 np.int64(84000),
 np.int64(84350),
 np.int64(85000),
 np.int64(86000),
 np.int64(86800),
 np.int64(87000),
 np.int64(87900),
 np.int64(88000),
 np.int64(89000),
 np.int64(89500),
 np.int64(90000),
 np.int64(91500),
 np.int64(92000),
 np.int64(92400),
 np.int64(93000),
 np.int64(94000),
 np.int64(94900),
 np.int64(95000),
 np.int64(95156),
 np.int64(95900),
 np.int64(96000),
 np.int64(97000),
 np.int64(97400),
 np.int64(

In [38]:
copia.to_json("datos_geolocalizados_zonaprop.json", orient="records", force_ascii=False)